In [1]:
127.0.0.1 ####This is the IP address (or host name, if available) of the client (remote host) which made the request to the server.

- ####The "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.

- ####The "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available.

[01/Aug/1995:00:00:01 -0400] ####The time that the server finished processing the request. The format is: [day/month/year:hour:minute:second timezone]

    ####day = 2 digits
    ####month = 3 letters
    ####year = 4 digits
    ####hour = 2 digits
    ####minute = 2 digits
    ####second = 2 digits
    ####zone = (+ | -) 4 digits

"GET /images/launch-logo.gif HTTP/1.0" ####This is the first line of the request string from the client. It consists of a three components: the request method (e.g., GET, POST, etc.), the endpoint (a Uniform Resource Identifier), and the client protocol version.

200 ####This is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification (RFC 2616 section 10).

1839 ####The last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).



SyntaxError: invalid syntax (<ipython-input-1-cb5ecfedc3b4>, line 1)

In [2]:
logtest = 'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'

In [45]:
import re
import datetime
from pyspark.sql import Row, SQLContext

In [26]:
REGEXPAPACHE = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)\s*" (\d{3}) (\S+)'
m = re.search(REGEXPAPACHE, logtest)

In [27]:
m.group(9)

'1839'

In [28]:
apachelogRDD = sc.textFile('./apache.access.log.PROJECT')

In [29]:
def parseTime(dateText):
    return datetime.datetime()

In [30]:
def parseline(logline):
    match = re.search(REGEXPAPACHE, logline)
    
    #if not match skip
    if match is None:
        return (logline,0)
    
    #Get size info as number
    size_field = match.group(9)
    if size_field == '-':
        size_field = 0
    else:
        size_field = long(size_field)
    
    return (Row(
        host = match.group(1),
        client = match.group(2),
        userinfo = match.group(3),
        date_time = match.group(4),
        method = match.group(5),
        endpoint = match.group(6),
        protocol = match.group(7),
        response_code = match.group(8),
        content_size = size_field
            
        ), 1)

In [31]:
month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[7:11]),
                             month_map[s[3:6]],
                             int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))

In [38]:
apacheParsedLogRDD = apachelogRDD.map(lambda e: parseline(e))

In [39]:
apacheParsedLogRDD.take(10)

[(Row(client=u'-', content_size=1839L, date_time=u'01/Aug/1995:00:00:01 -0400', endpoint=u'/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', host=u'in24.inetnebr.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'200', userinfo=u'-'),
  1),
 (Row(client=u'-', content_size=0L, date_time=u'01/Aug/1995:00:00:07 -0400', endpoint=u'/', host=u'uplherc.upl.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'304', userinfo=u'-'),
  1),
 (Row(client=u'-', content_size=0L, date_time=u'01/Aug/1995:00:00:08 -0400', endpoint=u'/images/ksclogo-medium.gif', host=u'uplherc.upl.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'304', userinfo=u'-'),
  1),
 (Row(client=u'-', content_size=0L, date_time=u'01/Aug/1995:00:00:08 -0400', endpoint=u'/images/MOSAIC-logosmall.gif', host=u'uplherc.upl.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'304', userinfo=u'-'),
  1),
 (Row(client=u'-', content_size=0L, date_time=u'01/Aug/1995:00:00:08 -0400', endpoint=u'/images/USA-logos

In [42]:
goodLogRDD = apacheParsedLogRDD.filter(lambda s: s[1]==1).map(lambda e: e[0])

In [43]:
goodLogRDD.take(3)

[Row(client=u'-', content_size=1839L, date_time=u'01/Aug/1995:00:00:01 -0400', endpoint=u'/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', host=u'in24.inetnebr.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'200', userinfo=u'-'),
 Row(client=u'-', content_size=0L, date_time=u'01/Aug/1995:00:00:07 -0400', endpoint=u'/', host=u'uplherc.upl.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'304', userinfo=u'-'),
 Row(client=u'-', content_size=0L, date_time=u'01/Aug/1995:00:00:08 -0400', endpoint=u'/images/ksclogo-medium.gif', host=u'uplherc.upl.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'304', userinfo=u'-')]

In [44]:
sc.version

u'1.5.2'

In [46]:
sqlContext = SQLContext(sc)

In [47]:
schemaLog = sqlContext.createDataFrame(goodLogRDD)

In [49]:
schemaLog.registerTempTable("logs")

In [50]:
result = sqlContext.sql("SELECT * FROM logs LIMIT 1")

In [51]:
result.collect()

[Row(client=u'-', content_size=1839, date_time=u'01/Aug/1995:00:00:01 -0400', endpoint=u'/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', host=u'in24.inetnebr.com', method=u'GET', protocol=u'HTTP/1.0', response_code=u'200', userinfo=u'-')]

In [66]:
result = sqlContext.sql("SELECT host,COUNT(*) as total FROM logs GROUP BY host ORDER BY total DESC LIMIT 10")

In [67]:
result.collect()

[Row(host=u'edams.ksc.nasa.gov', total=1013),
 Row(host=u'piweba5y.prodigy.com', total=871),
 Row(host=u'piweba3y.prodigy.com', total=824),
 Row(host=u'163.205.156.16', total=748),
 Row(host=u'www-a2.proxy.aol.com', total=746),
 Row(host=u'piweba4y.prodigy.com', total=702),
 Row(host=u'www-d3.proxy.aol.com', total=659),
 Row(host=u'www-relay.pa-x.dec.com', total=657),
 Row(host=u'www-b4.proxy.aol.com', total=656),
 Row(host=u'news.ti.com', total=655)]